In [1]:
pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client

In [2]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [3]:
import os
os.listdir('/content')



['.config',
 'credentials (1).json',
 'token.json',
 'credentials.json',
 'sample_data']

In [25]:
# 1. SETUP GMAIL AUTHENTICATION
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0, open_browser=False) # Modified to not open browser
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()

In [26]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credential.json',
                SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'
            )

            auth_url, _ = flow.authorization_url(
                prompt='consent'
            )

            print("\n🔗 OPEN THIS URL IN YOUR BROWSER:\n")
            print(auth_url)

            code = input("\n📌 Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()

In [27]:
def read_inbox(service, max_results=5):
    results = service.users().messages().list(
        userId='me',
        labelIds=['INBOX'],
        maxResults=max_results
    ).execute()

    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    for msg in messages:
        msg_id = msg['id']
        message = service.users().messages().get(
            userId='me',
            id=msg_id,
            format='metadata',
            metadataHeaders=['From', 'Subject']
        ).execute()

        headers = message['payload']['headers']
        email_data = {}

        for h in headers:
            email_data[h['name']] = h['value']

        print("\n📩 New Email")
        print("From:", email_data.get('From'))
        print("Subject:", email_data.get('Subject'))

In [29]:
read_inbox(service)


📩 New Email
From: Team Unstop <noreply@unstop.news>
Subject: Time’s slipping away: Hero Campus Challenge Season 10: Apply now!

📩 New Email
From: Naukri Campus <eventnc@naukri.com>
Subject: Bhavya,  redBus invites you to play & win

📩 New Email
From: Streaming Panel <supportstreamingpanel@nielsen.com>
Subject: Nielsen Streaming Panel - Daily Points Have Been Added

📩 New Email
From: SmartBridge <info@thesmartbridge.com>
Subject:  🚀 Get Ready to Build! Hack-N-Go with MongoDB – Bootcamp Details

📩 New Email
From: Aryan from Unstop <noreply@dare2compete.news>
Subject: You’re close, take the final step


In [32]:
from openai import OpenAI

client = OpenAI(api_key="Open_api_key")

print("Available OpenAI models:")
for model in client.models.list():
    print(model.id)

Available OpenAI models:
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-i

In [39]:
 # Checked if Gmail is connected
profile = service.users().getProfile(userId='me').execute()
print("Email:", profile["emailAddress"])
print("Total messages:", profile["messagesTotal"])

Email: bhavyamahali@gmail.com
Total messages: 8270


In [40]:
results = service.users().messages().list(
    userId = 'me',
    maxResults = 2
).execute()

messages = results.get('messages', [])
print(messages)

[{'id': '19be54d3e79ca0b9', 'threadId': '19be54d3e79ca0b9'}, {'id': '19be527ccc47e2d6', 'threadId': '19be527ccc47e2d6'}]


In [41]:
def get_message(service, msg_id):
    return service.users().messages().get(
        userId = 'me',
        id = msg_id,
        format = 'full'
    ).execute()

msg = get_message(service, messages[0]['id'])
print(msg.keys())
print(msg['snippet'])

dict_keys(['id', 'threadId', 'labelIds', 'snippet', 'payload', 'sizeEstimate', 'historyId', 'internalDate'])
From campus to Hero careers ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿ ͏ ﻿


In [42]:
def extract_email_data(msg):
    headers = msg['payload']['headers']
    data = {}

    for h in headers:
        name = h['name']
        if name in ['From', 'To', 'Subject', 'Date']:
            data[name] = h['value']

    parts = msg['payload'].get('parts')
    body = ""

    if parts:
        for part in parts:
            if part['mimeType'] == 'text/plain':
                body = base64.urlsafe_b64decode(
                    part['body']['data']
                ).decode("utf-8", errors="ignore")
    else:
        body = base64.urlsafe_b64decode(
            msg['payload']['body']['data']
        ).decode("utf-8", errors="ignore")

    data["body"] = body
    return data


email_data = extract_email_data(msg)
print(email_data)

{'Subject': 'Time’s slipping away: Hero Campus Challenge Season 10: Apply now!', 'From': 'Team Unstop <noreply@unstop.news>', 'To': 'bhavyamahali@gmail.com', 'Date': 'Thu, 22 Jan 2026 16:13:23 +0530', 'body': ''}


In [43]:
for m in messages:
    msg = get_message(service, m['id'])
    email = extract_email_data(msg)

    print("\n----Mail----\n")
    print("From:", email.get("From"))
    print("Subject:", email.get("Subject"))
    print("Body:", email["body"][:500])


----Mail----

From: Team Unstop <noreply@unstop.news>
Subject: Time’s slipping away: Hero Campus Challenge Season 10: Apply now!
Body: 

----Mail----

From: Naukri Campus <eventnc@naukri.com>
Subject: Bhavya,  redBus invites you to play & win
Body: 
